In [ ]:
import cv2
import glob
import random
import numpy as np
import pandas as pd
import albumentations as A
import matplotlib.pyplot as plt
import segmentation_models as sm

from dataset import Dataset
from dataset import visualize
from dataset import Dataloader
from dataset import denormalize
from keras.models import load_model
from segmentation_models.losses import DiceLoss
from segmentation_models.losses import CategoricalFocalLoss
from segmentation_models.metrics import IOUScore
from segmentation_models.metrics import FScore

In [ ]:
CATEGORY_PATH = "dataset/category_id.txt"
TEST_PATH = "dataset/test/"
TEST_DIRS = glob.glob("dataset/test/*")

MODEL = "fpn-seresnet152"
CHECKPOINT_PATH = "checkpoints/" + MODEL + ".h5"
MODEL_PATH = "models/" + MODEL + ".h5"

THRESHOLD = 0.9

In [ ]:
def get_classes():
    categories = pd.read_csv(CATEGORY_PATH, sep="\t", names=["id", "name"])
    # CamVid Dataset
    # ids = categories["id"].to_list()[:-1]
    # classes = categories["name"].to_list()[:-1]
    # FoodSeg103 Dataset
    ids = categories["id"].to_list()[1:]
    classes = categories["name"].to_list()[1:]
    classes = [cls.strip() for cls in classes]
    return ids, classes

In [ ]:
def get_val_test_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [A.PadIfNeeded(384, 480)]
    return A.Compose(test_transform)

In [ ]:
def get_preprocessing(preprocessing_fn):
    _transform = [A.Lambda(image=preprocessing_fn)]
    return A.Compose(_transform)

In [ ]:
preprocess_input = sm.get_preprocessing(MODEL.split("-")[1])

In [ ]:
ids, classes = get_classes()

In [ ]:
test_dataset = Dataset(TEST_PATH + "img", TEST_PATH + "mask", class_values=ids,
                       augmentation=get_val_test_augmentation(),
                       preprocessing=get_preprocessing(preprocess_input))

In [ ]:
test_dataloader = Dataloader(test_dataset, batch_size=1, shuffle=False)
print(test_dataloader[0][0].shape)
print(test_dataloader[0][1].shape)

In [ ]:
dice_loss = DiceLoss(class_weights=np.append(np.ones(len(get_classes()[0])), 0.5))
focal_loss = CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)
iou_score = IOUScore(threshold=0.5)
f1_score = FScore(threshold=0.5)

In [ ]:
custom_objects = {
    "iou_score": iou_score,
    "f1-score": f1_score,
    "dice_loss_plus_1focal_loss": total_loss
}
model = load_model(CHECKPOINT_PATH, custom_objects=custom_objects)
print("Model Loaded")

In [ ]:
test_score = model.evaluate_generator(test_dataloader)
print("Test Loss: ", test_score[0])
print("Test Mean IoU: ", test_score[1])
print("Test Mean F1: ", test_score[2])

In [ ]:
# CamVid Dataset
# image, mask = test_dataset[0]
# visualize(image=image,
#           sky_mask=mask[..., 0].squeeze(),
#           car_mask=mask[..., 8].squeeze(),
#           pedestrian_mask=mask[..., 9].squeeze(),
#           unlabelled_mask=mask[..., 11].squeeze())

In [ ]:
# CamVid Dataset
# n = 5
# ids = np.random.choice(np.arange(len(test_dataset)), size=n)
# for i in ids:
#     image, gt_mask = test_dataset[i]
#     image = np.expand_dims(image, axis=0)
#     pr_mask = model.predict(image)
#     visualize(
#         image=denormalize(image.squeeze()),
#         gt_sky_mask=gt_mask[..., 0].squeeze(),
#         pr_sky_mask=pr_mask[..., 0].squeeze(),
#         gt_car_mask=gt_mask[..., 8].squeeze(),
#         pr_car_mask=pr_mask[..., 8].squeeze())

In [ ]:
# FoodSeg103 Dataset
image, mask = test_dataset[0]
visualize(image=denormalize(image.squeeze()),
          shrimp_mask=mask[..., 55].squeeze(),
          rice_mask=mask[..., 65].squeeze(),
          carrot_mask=mask[..., 83].squeeze(),
          french_beans_mask=mask[..., 95].squeeze(),
          background_mask=mask[..., 103].squeeze())

In [ ]:
image, gt_mask = test_dataset[0]
image = np.expand_dims(image, axis=0)
pr_mask = np.where((model.predict(image)) < THRESHOLD, 0, 1)
visualize(
    image=denormalize(image.squeeze()),
    gt_shrimp_mask=gt_mask[..., 55].squeeze(),
    pr_shrimp_mask=pr_mask[..., 55].squeeze(),
    gt_rice_mask=gt_mask[..., 65].squeeze(),
    pr_rice_mask=pr_mask[..., 65].squeeze())
visualize(
    image=denormalize(image.squeeze()),
    gt_carrot_mask=gt_mask[..., 83].squeeze(),
    pr_carrot_mask=pr_mask[..., 83].squeeze(),
    gt_french_beans_mask=gt_mask[..., 95].squeeze(),
    pr_french_beans_mask=pr_mask[..., 95].squeeze())

In [ ]:
gt_img = np.zeros((384, 480))
for i in range(len(ids) + 1):
    tmp = gt_mask[..., i].squeeze()
    gt_img[tmp == 1] = i
gt_img = cv2.merge((gt_img, gt_img, gt_img))
plt.subplot(1, 2, 1)
plt.imshow(gt_img.astype("uint8"))
plt.axis("off")

pr_class_indices = []
pr_img = np.zeros((384, 480))
for i in range(len(ids) + 1):
    tmp = pr_mask[..., i].squeeze()
    if np.count_nonzero(tmp == 1) > 0:
        pr_class_indices.append(i)
    pr_img[tmp == 1] = i
pr_img = cv2.merge((pr_img, pr_img, pr_img))
plt.subplot(1, 2, 2)
plt.imshow(pr_img.astype("uint8"))
plt.axis("off")

plt.tight_layout()

print(pr_class_indices)